In [58]:
import pandas as pd
import numpy as np
import random

np.random.seed(1)
random.seed(1)

In [59]:
TRAININFO = "data/train_info.tsv"
TRAINRANK =  "data/train_rank.csv"
TESTINFO = "data/test_info.tsv"
Track_info = "data/all_track_info.csv"
Audio_info = "data/all_track_audio_features.csv"

In [60]:
df_i = pd.read_csv(TRAININFO, delimiter='\t',encoding='utf-8')
df_r = pd.read_csv(TRAINRANK)
df_i_train = df_i.merge(df_r, left_on='ID', right_on='ID')
df_i_train["dataset"] = "train"

df_i_test = pd.read_csv(TESTINFO, delimiter='\t',encoding='utf-8')
df_i_test["label"] = np.nan
df_i_test["dataset"] = "test"

df = pd.concat([df_i_train, df_i_test])
df_track_info = pd.read_csv(Track_info)
df = df.merge(df_track_info, left_on='ID', right_on='ID')
df_audio_features = pd.read_csv(Audio_info)
df =df.merge(df_audio_features,left_on="ID",right_on="ID", how="left")
df = df[['ID','title','artist_name','lyric','label','dataset']]
df.head()

,ID,title,artist_name,lyric,label,dataset
0,1073748245,Đêm Chôn Dầu Vượt Biển,Như Quỳnh,Đêm nay anh gánh dầu ra biển anh chôn \r\nAnh ...,7.0,train
1,1073751978,Mùa Thu Trong Mưa,Minh Tuyết,Chiều mưa không có em\r\nbờ đá công viên âm th...,3.0,train
2,1073835561,Rồi Ánh Trăng Tan,Lưu Bích,Rồi ánh trăng cũng đang tan dần\r\nRồi ước mơ ...,6.0,train
3,1073856553,Còn Thương Rau Đắng Mọc Sau Hè,Như Quỳnh,Nắng hạ đi Mây trôi lang thang cho hạ buồn Coi...,2.0,train
4,1073929630,Người Điên Biết Yêu,Như Loan,"Ai trong tình yêu, ai không mơ mộng?\n\nNgu ng...",7.0,train


In [61]:
import re

df.drop([6707,6718,7426,7362,8499,8500,7360,7361,7214], inplace=True)
df.lyric.fillna("", inplace=True)
def cleanString(string):
  return re.sub('\W+',' ', string )
df.lyric = df.lyric.apply(lambda x: cleanString(x))
df.lyric = df.lyric.apply(lambda x: x.lower())

def artist_func(string):
  return ''.join(string.split(' '))+ "_"
df['artist_name'] = df['artist_name'].apply(lambda x: artist_func(x))
df['lyric'] = df['artist_name'] + df['lyric']

def personalized(string):
  lst = string.split('_')[1].split(" ")
  for i in range(0,len(lst)):
    lst[i] = string.split("_")[0]+"_"+lst[i]
  return " ".join(lst)
df['lyric'] = df['lyric'].apply(lambda x: personalized(x))

df_train=df[df.dataset=='train']
print(df_train.shape)
df_train.drop_duplicates(subset='lyric', keep='first',inplace=True)
print(df_train.shape)
df_train.head()

(9069, 6)
(4594, 6)


/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ID,title,artist_name,lyric,label,dataset
0,1073748245,Đêm Chôn Dầu Vượt Biển,NhưQuỳnh_,NhưQuỳnh_đêm NhưQuỳnh_nay NhưQuỳnh_anh NhưQuỳn...,7.0,train
1,1073751978,Mùa Thu Trong Mưa,MinhTuyết_,MinhTuyết_chiều MinhTuyết_mưa MinhTuyết_không ...,3.0,train
2,1073835561,Rồi Ánh Trăng Tan,LưuBích_,LưuBích_rồi LưuBích_ánh LưuBích_trăng LưuBích_...,6.0,train
3,1073856553,Còn Thương Rau Đắng Mọc Sau Hè,NhưQuỳnh_,NhưQuỳnh_nắng NhưQuỳnh_hạ NhưQuỳnh_đi NhưQuỳnh...,2.0,train
4,1073929630,Người Điên Biết Yêu,NhưLoan_,NhưLoan_ai NhưLoan_trong NhưLoan_tình NhưLoan_...,7.0,train


In [62]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

# def ex_features(train_set,hash=False):
#     if hash:
#         vectorizer = HashingVectorizer(non_negative=True, n_features = 2**9)
#     else:
#         vectorizer = TfidfVectorizer()
#     vectors = vectorizer.fit_transform(train_set.lyric)
#     dense = vectors.todense()
#     skf = StratifiedKFold(n_splits=5, random_state=99999)
#     params = {}
# # denselist = dense.tolist()
# # X_train = pd.DataFrame(denselist, columns=feature_names)
#     nb = MultinomialNB()
#     gs = GridSearchCV(nb, cv=skf, param_grid=params, scoring='mean_squared_error')
#     gs.fit(dense, train_set.label)
#     return np.sqrt(mse(gs.predict(dense), train_set.label))

In [65]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df_train.lyric)
dense = vectors.todense()
skf = StratifiedKFold(n_splits=5, random_state=99999)
params = {}
# denselist = dense.tolist()
# X_train = pd.DataFrame(denselist, columns=feature_names)
nb = MultinomialNB()
gs = GridSearchCV(nb, cv=skf, param_grid=params, scoring='neg_mean_squared_error',return_train_score=True)
gs.fit(dense, df_train.label)
print(gs.cv_results_)
print("best score: "+str(np.sqrt(-(gs.best_score_))))

/usr/lib64/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


{'mean_fit_time': array([5.06885529]), 'std_fit_time': array([1.59133743]), 'mean_score_time': array([2.08281956]), 'std_score_time': array([0.08474047]), 'params': [{}], 'split0_test_score': array([-24.54821235]), 'split1_test_score': array([-21.15635179]), 'split2_test_score': array([-18.0785169]), 'split3_test_score': array([-16.21592148]), 'split4_test_score': array([-20.53930131]), 'mean_test_score': array([-20.11427949]), 'std_test_score': array([2.83978414]), 'rank_test_score': array([1], dtype=int32), 'split0_train_score': array([-1.4083356]), 'split1_train_score': array([-1.3299755]), 'split2_train_score': array([-1.03725863]), 'split3_train_score': array([-0.99211314]), 'split4_train_score': array([-0.99537792]), 'mean_train_score': array([-1.15261216]), 'std_train_score': array([0.17924353])}
best score: 4.484894591291246


In [66]:
vectorizer = HashingVectorizer(non_negative=True, n_features = 2**9)
vectors = vectorizer.fit_transform(df_train.lyric)
dense = vectors.todense()
skf = StratifiedKFold(n_splits=5, random_state=99999)
params = {}
nb = MultinomialNB()
gs = GridSearchCV(nb, cv=skf, param_grid=params, scoring='neg_mean_squared_error',return_train_score=True)
gs.fit(dense, df_train.label)
print(gs.cv_results_)
print("best score: "+str(np.sqrt(-(gs.best_score_))))

/usr/lib64/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/usr/lib64/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


{'mean_fit_time': array([0.00969534]), 'std_fit_time': array([0.00025366]), 'mean_score_time': array([0.00454626]), 'std_score_time': array([1.62563012e-05]), 'params': [{}], 'split0_test_score': array([-21.36511376]), 'split1_test_score': array([-21.59500543]), 'split2_test_score': array([-18.58342421]), 'split3_test_score': array([-15.74154853]), 'split4_test_score': array([-19.51965066]), 'mean_test_score': array([-19.36547671]), 'std_test_score': array([2.13263964]), 'rank_test_score': array([1], dtype=int32), 'split0_train_score': array([-12.83873604]), 'split1_train_score': array([-13.13095562]), 'split2_train_score': array([-12.86374762]), 'split3_train_score': array([-12.31302692]), 'split4_train_score': array([-13.18814573]), 'mean_train_score': array([-12.86692239]), 'std_train_score': array([0.31000088])}
best score: 4.400622309259288
